#  Evaluating a Base LLM for Summarization (Baseline Tutorial)

This notebook walks through the process of establishing a *baseline* performance for a large language model (LLM) on a summarization task.

**Goal:** To understand how the original, un-finetuned model performs so we have a benchmark to compare against after fine-tuning.

**Task:** Summarizing the [SAMSum Dataset](https://huggingface.co/datasets/knkarthick/samsum).

**Metric:** We will use the ROUGE score to evaluate the quality of the generated summaries.

## 1. 📚 Setup: Install Dependencies

First, we need to install all the required Python libraries.

* `evaluate`: Hugging Face's library for model evaluation (used for ROUGE).
* `torch`: The core PyTorch library.
* `datasets`: For loading the SAMSum dataset.
* `peft`: Parameter-Efficient Fine-Tuning (used for LoRA).
* `transformers`: For loading the model and tokenizer.
* `bitsandbytes`: For quantization (which we will disable for this baseline but is needed for the helper functions).
* `rouge_score`: The specific package needed by `evaluate` for ROUGE.

In [ ]:
# Install required libraries for Hugging Face, evaluation, and PEFT
! pip install -q evaluate torch tqdm datasets peft transformers rouge_score

# Install bitsandbytes for quantization support
! pip install -U bitsandbytes

## 2. 📦 Import Libraries

Now, let's import all the modules we'll use throughout the notebook.

In [ ]:
import os
import yaml  # For loading the configuration file
import json  # For saving results
import torch
import evaluate  # For loading the ROUGE metric
from tqdm import tqdm  # For progress bars
from datasets import load_dataset, load_from_disk

# PEFT (Parameter-Efficient Fine-Tuning) libraries
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Transformers libraries for model, tokenizer, and quantization
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

In [ ]:
# Define a constant for the dataset directory (if loading from disk)
DATASETS_DIR = "./datasets"

## 3. 💾 Data Loading and Preparation

We'll define two helper functions to manage our dataset.

* `select_subset`: A utility to select a specific number of samples from a dataset split (e.g., "all" or 200). This is useful for running quick experiments.
* `load_and_prepare_dataset`: The main function that loads the dataset from Hugging Face (based on the config) and uses `select_subset` to prepare our `train`, `val`, and `test` splits.

In [ ]:
def select_subset(dataset, n_samples, seed=42):
    """
    Select a subset of the dataset.
    If n_samples is "all" or None, return the entire dataset.
    Otherwise, sample n_samples examples.
    """
    if n_samples == "all" or n_samples is None:
        return dataset

    if n_samples > len(dataset):
        print(f"⚠️  Requested {n_samples} samples but only {len(dataset)} available. Using all samples.")
        return dataset

    return dataset.shuffle(seed=seed).select(range(n_samples))


def load_and_prepare_dataset(cfg):
    """
    Load dataset splits according to configuration.
    """
    cfg_dataset = cfg["dataset"]
    dataset = load_dataset(cfg_dataset["name"])
    val_key = "validation" if "validation" in dataset else "val"

    train = select_subset(dataset["train"], cfg_dataset["splits"]["train"], seed=42)
    val = select_subset(dataset[val_key], cfg_dataset["splits"]["validation"], seed=42)
    test = select_subset(dataset["test"], cfg_dataset["splits"]["test"], seed=42)

    print(f"📊 Loaded {len(train)} train / {len(val)} val / {len(test)} test samples.")
    return train, val, test

## 4. 🤖 Model & Tokenizer Setup

This is a crucial helper function responsible for loading the model and tokenizer. It's designed to be flexible and can handle three main scenarios:

1.  **Full Precision (Baseline):** Loads the model as-is (e.g., in `bfloat16`). This is what we'll use for our baseline.
2.  **Quantized (QLoRA Prep):** Loads the model in 4-bit precision using `BitsAndBytesConfig`. This is the 'Q' in QLoRA, a technique to reduce memory usage.
3.  **Quantized + LoRA:** Applies LoRA (Low-Rank Adaptation) adapters to the quantized model, making it ready for PEFT (Parameter-Efficient Fine-Tuning).

For this tutorial, we will force it into **scenario 1** by passing `use_4bit=False` and `use_lora=False` in the final step.

In [ ]:
def setup_model_and_tokenizer(cfg, use_4bit: bool = None, use_lora: bool = None):
    """
    Load model, tokenizer, and apply quantization + LoRA config if specified.

    Args:
        cfg (dict): Configuration dictionary containing:
            - base_model
            - quantization parameters
            - lora parameters (optional)
            - bf16 or fp16 precision
        use_4bit (bool, optional): Override whether to load in 4-bit mode.
        use_lora (bool, optional): Override whether to apply LoRA adapters.

    Returns:
        tuple: (model, tokenizer)
    """
    model_name = cfg["base_model"]
    print(f"\nLoading model: {model_name}")

    # ------------------------------
    # Tokenizer setup
    # ------------------------------
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    # Determine quantization + LoRA usage
    # Use override if provided, otherwise default to config file
    load_in_4bit = use_4bit if use_4bit is not None else cfg.get("load_in_4bit", False)
    apply_lora = use_lora if use_lora is not None else ("lora_r" in cfg)

    # ------------------------------
    # Quantization setup
    # ------------------------------
    quant_cfg = None
    if load_in_4bit:
        print("⚙️  Enabling 4-bit quantization (QLoRA)...")
        quant_cfg = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type=cfg.get("bnb_4bit_quant_type", "nf4"),
            bnb_4bit_use_double_quant=cfg.get("bnb_4bit_use_double_quant", True),
            bnb_4bit_compute_dtype=getattr(
                torch, cfg.get("bnb_4bit_compute_dtype", "bfloat16")
            ),
        )
    else:
        print("⚙️  Loading model in full precision (no quantization).")

    # ------------------------------
    # Model loading
    # ------------------------------
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quant_cfg, # Will be None if not 4-bit
        device_map="auto", # Automatically spread model across GPUs
        dtype=(
            torch.bfloat16
            if cfg.get("bf16", True) and torch.cuda.is_available()
            else torch.float32
        ),
    )

    # ------------------------------
    # LoRA setup (optional)
    # ------------------------------
    if apply_lora:
        print("🔧 Applying LoRA configuration...")
        # Prepare model for k-bit training if quantization is enabled
        if load_in_4bit:
            model = prepare_model_for_kbit_training(model)
        
        lora_cfg = LoraConfig(
            r=cfg.get("lora_r", 8),
            lora_alpha=cfg.get("lora_alpha", 16),
            target_modules=cfg.get("target_modules", ["q_proj", "v_proj"]),
            lora_dropout=cfg.get("lora_dropout", 0.05),
            bias="none",
            task_type="CAUSAL_LM",
        )
        model = get_peft_model(model, lora_cfg)
        model.print_trainable_parameters()
    else:
        print("🔹 Skipping LoRA setup — using base model only.")

    return model, tokenizer

## 5. ⚙️ Configuration

We use a central `config.yaml` file to manage all our settings, such as the model name, dataset, and LoRA parameters. This makes our experiments reproducible and easy to modify.

### ❗ Action Required: Create `config.yaml`

You must create a file named `config.yaml` in the **same directory** as this notebook. Copy and paste the following content into it.

This configuration is based on the file you provided and sets up the model (`meta-llama/Llama-3.2-1B-Instruct`), dataset (`knkarthick/samsum`), and various training parameters (which we won't use in *this* notebook, but are needed for the file to load).

```yaml
# ==============================================
# Model Configuration
# ==============================================
base_model: meta-llama/Llama-3.2-1B-Instruct
tokenizer_type: meta-llama/Llama-3.2-1B-Instruct

# ==============================================
# Dataset
# ==============================================
dataset:
  name: knkarthick/samsum
  cache_dir: ../data/datasets
  field_map:
    input: dialogue
    output: summary
  type: completion
  splits:
    train: all
    validation: 200
    test: 200
  seed: 42

# This is for Axolotl
datasets:
  - path: knkarthick/samsum
    cache_dir: ../data/datasets
    field_map:
      input: dialogue
      output: summary
    type: completion

task_instruction: >
  You are a helpful assistant who writes concise, factual summaries of conversations.
  Summarize the following conversation into a single sentence.

train_samples: all
val_samples: 200
test_samples: 200
seed: 42

# ==============================================
# Quantization (for QLoRA)
# ==============================================
load_in_4bit: true
bnb_4bit_quant_type: nf4
bnb_4bit_use_double_quant: true
bnb_4bit_compute_dtype: bfloat16

# ==============================================
# LoRA Configuration
# ==============================================
lora_r: 16
lora_alpha: 32
lora_dropout: 0.1
target_modules: ["q_proj", "v_proj"]

# ==============================================
# Training Configuration
# ==============================================
num_epochs: 1
max_steps: 300
learning_rate: 2e-4
batch_size: 4
gradient_accumulation_steps: 4
sequence_len: 512
lr_scheduler: cosine
warmup_steps: 50
bf16: true
logging_steps: 25
save_steps: 100
save_total_limit: 2
optim: paged_adamw_8bit

# ==============================================
# Output & Logging
# ==============================================
output_dir: ./outputs/lora_samsum
wandb_project: llama3_samsum
wandb_run_name: lora-finetuning-default-hps
```

In [ ]:
# Define the path to our configuration file
CONFIG_FILE_PATH = "./config.yaml"

def load_config(config_path: str = CONFIG_FILE_PATH):
    """
    Load and parse a YAML configuration file.

    Args:
        config_path (str): Path to the config file.

    Returns:
        dict: Parsed configuration dictionary.
    """
    # Add a check to ensure the file exists
    if not os.path.exists(config_path):
        raise FileNotFoundError(
            f"Configuration file not found at: {config_path}. "
            "Please create it using the template in the Markdown cell above."
        )
        
    with open(config_path, "r", encoding="utf-8") as f:
        cfg = yaml.safe_load(f)
    return cfg

## 6. 📈 Evaluation Functions

Here we define the core logic for our evaluation.

### `generate_predictions`

This function takes the model, tokenizer, and dataset, then:
1.  Formats each 'dialogue' into a chat prompt using the `task_instruction` from our config.
2.  Uses a Hugging Face `pipeline` for efficient, batched text generation.
3.  Collects all the generated 'summaries' (the predictions).

### `compute_rouge`

This function calculates the ROUGE score, a standard metric for summarization.

**What is ROUGE?** ROUGE (Recall-Oriented Understudy for Gisting Evaluation) measures the overlap between the model-generated summary and a human-written 'reference' summary.
* **ROUGE-1:** Measures overlap of unigrams (single words).
* **ROUGE-2:** Measures overlap of bigrams (pairs of words).
* **ROUGE-L:** Measures the longest common subsequence (LCS), which checks for sentence-level structure.

In [ ]:
def generate_predictions(
    model,
    tokenizer,
    dataset,
    task_instruction,
    num_samples=None,
    batch_size=8,
    max_new_tokens=256,
):
    """
    Generate model predictions for a dataset (e.g., summaries).

    Args:
        model: The loaded model (base or fine-tuned).
        tokenizer: Corresponding tokenizer.
        dataset: Hugging Face dataset split containing 'dialogue' and 'summary'.
        task_instruction (str): Instruction prefix for generation.
        num_samples (int, optional): Number of samples to evaluate.
        batch_size (int): Number of examples per inference batch.
        max_new_tokens (int): Max tokens to generate per sample.

    Returns:
        list[str]: Generated summaries.
    """
    # Select a subset of the dataset if num_samples is specified
    if num_samples is not None and num_samples < len(dataset):
        dataset = dataset.select(range(num_samples))

    # Prepare prompts
    prompts = []
    for sample in dataset:
        # Format the input using the instruction and dialogue
        user_prompt = (
            f"{task_instruction}\n\n"
            f"## Dialogue:\n{sample['dialogue']}\n"
            "## Summary:"
        )
        # Use the chat template for the specific model (e.g., Llama 3's)
        messages = [{"role": "user", "content": user_prompt}]
        prompt = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        prompts.append(prompt)

    # Initialize the text-generation pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        dtype="auto", # Use model's native dtype (e.g., bfloat16)
        do_sample=False, # Use greedy decoding for deterministic output
    )

    # Generate predictions in batches for efficiency
    preds = []
    for i in tqdm(range(0, len(prompts), batch_size), desc="Generating summaries"):
        batch = prompts[i : i + batch_size]
        # 'return_full_text=False' ensures we only get the generated summary,
        # not the prompt + summary
        outputs = pipe(batch, max_new_tokens=max_new_tokens, return_full_text=False)
        preds.extend([o[0]["generated_text"].strip() for o in outputs])

    return preds


def compute_rouge(predictions, samples):
    """
    Compute ROUGE scores between predictions and reference summaries.

    Args:
        predictions (list[str]): Model-generated outputs.
        samples (datasets.Dataset): Dataset containing reference 'summary' field.

    Returns:
        dict: ROUGE-1, ROUGE-2, and ROUGE-L scores.
    """
    # Load the ROUGE metric from the 'evaluate' library
    rouge = evaluate.load("rouge")
    # Get the ground-truth summaries from the dataset
    references = [s["summary"] for s in samples]
    # Compute the scores
    return rouge.compute(predictions=predictions, references=references)

## 7. 🚀 Run Baseline Evaluation

It's time to put everything together!

Here's the plan:
1.  **Load Config:** Read the `config.yaml` file.
2.  **Load Data:** Load the validation split of the SAMSum dataset.
3.  **Load Model:** Call our `setup_model_and_tokenizer` function.
    * **Crucially**, we will pass `use_4bit=False` and `use_lora=False`. This tells the function to ignore the quantization and LoRA settings in the config file and load the **true, original base model** in its standard precision (e.g., bfloat16). This is our baseline.
4.  **Generate:** Run the model over the validation set to get predictions.
5.  **Score:** Compute the ROUGE scores for the predictions.
6.  **Save & Print:** Save the results to JSON files and print a summary to the screen.

In [ ]:
"""
evaluate_baseline.py
Evaluate the base (unfine-tuned) model on the SAMSum dataset to establish baseline ROUGE scores.
"""

# Load the global configuration from config.yaml
cfg = load_config()

def evaluate_baseline():
    """Run baseline evaluation on the SAMSum dataset using the base model."""

    # 1. Load validation data
    # We only need the validation split for this evaluation
    _, val_data, _ = load_and_prepare_dataset(cfg)

    # ----------------------------------------------------------------------
    # 2. Load model + tokenizer (IMPORTANT: Baseline Configuration)
    # ----------------------------------------------------------------------
    print("\nLoading the TRUE baseline model (full precision, no LoRA)...")
    model, tokenizer = setup_model_and_tokenizer(
        cfg=cfg,
        use_4bit=False,  # <-- Set to False to load the original model
        use_lora=False,  # <-- Set to False to skip adding LoRA adapters
    )

    # 3. Generate predictions
    preds = generate_predictions(
        model=model,
        tokenizer=tokenizer,
        dataset=val_data,
        task_instruction=cfg["task_instruction"],
        batch_size=4, # Adjust batch size based on your GPU memory
    )

    # 4. Compute ROUGE metrics
    scores = compute_rouge(preds, val_data)

    # ----------------------------------------------------------------------
    # 5. Save outputs
    # ----------------------------------------------------------------------
    results = {
        "model_name": cfg["base_model"] + " (Baseline)",
        "num_samples": len(val_data),
        "rouge1": scores["rouge1"],
        "rouge2": scores["rouge2"],
        "rougeL": scores["rougeL"],
    }

    # Define output file paths
    results_path = "eval_results_baseline.json"
    preds_path = "predictions_baseline.jsonl"

    # Save the summary scores
    with open(results_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2)

    # Save the detailed predictions (dialogue, reference, prediction)
    with open(preds_path, "w", encoding="utf-8") as f:
        for i, pred in enumerate(preds):
            json.dump(
                {
                    "dialogue": val_data[i]["dialogue"],
                    "reference": val_data[i]["summary"],
                    "prediction": pred,
                },
                f,
            )
            f.write("\n")

    print(f"\n💾 Saved results to: {results_path}")
    print(f"💾 Saved predictions to: {preds_path}")

    # Return all components needed for the final report
    return scores, preds, val_data


# --------------------------------------------------------------------------
# Main Execution Block
# --------------------------------------------------------------------------
if __name__ == "__main__":
    print("🚀 Starting baseline evaluation...")
    
    # Run the evaluation
    rouge_scores, predictions, val_samples = evaluate_baseline()
    
    print("\n✅ Evaluation complete.")


    print("\n📈 Baseline ROUGE Results:")
    # Print scores formatted as percentages
    print(f"  ROUGE-1: {rouge_scores['rouge1'] * 100:.2f}%")
    print(f"  ROUGE-2: {rouge_scores['rouge2'] * 100:.2f}%")
    print(f"  ROUGE-L: {rouge_scores['rougeL'] * 100:.2f}%")

    # Show an example of the model's output vs. the reference
    print("\n" + "="*50)
    print("🔍 Example Prediction (Sample 0)")
    print("="*50)
    print(f"\n[DIALOGUE]\n{val_samples[0]['dialogue']}\n")
    print(f"[REFERENCE SUMMARY]\n{val_samples[0]['summary']}\n")
    print(f"[MODEL PREDICTION]\n{predictions[0]}")
    print("="*50)

## 8. 🏁 Conclusion & Next Steps

Congratulations! You've successfully established a baseline ROUGE score for your model.

This score is your 'line in the sand.' It represents the model's out-of-the-box performance on this task.

### Why does this matter?

When you proceed to fine-tune this model (for example, using the QLoRA settings in your `config.yaml`), you can run this same evaluation script on your *new*, *fine-tuned* model.

By comparing the new ROUGE scores to this baseline, you can definitively measure exactly how much your fine-tuning improved the model's summarization ability.